In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd S:\PPMI_RNA\Processed\Progression_Analysis\Female\HYS_1vs2vs3
data = pd.read_csv('CountData_HYS1vs2vs3_PD_withHYS_Female.csv')
data = data[data['HYS'] != 3]
data.head(1)

S:\PPMI_RNA\Processed\Progression_Analysis\Female\HYS_1vs2vs3


,HudAlphaID,HYS,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,ENSG00000240361.2,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
0,5104_SL_0108,1,15,78,12,5,0,6,3,7,...,0,7,26,4,1,3,22,17,0,0


In [3]:
X = data.drop(['HudAlphaID', 'HYS'], axis = 1)
y = data['HYS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  ENSG00000243485.5  \
0                 15                 78                 12                  5   
1                  6                128                 19                  3   

   ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  ENSG00000240361.2  \
0                  0                  6                  3                  7   
1                  0                  3                  0                  2   

   ENSG00000186092.6  ENSG00000238009.6  ...  ENSG00000273739.1  \
0                  2                 35  ...                  0   
1                  0                 50  ...                  0   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
0                  7                 26                  4                  1   
1                  8                 30                 58                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.5135135135135135

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	58780


BorutaPy(estimator=RandomForestClassifier(n_estimators=3428,
                                          random_state=RandomState(MT19937) at 0x1DD795E4D40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1DD795E4D40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [8]:
selected_rf_features = pd.DataFrame({'GeneID':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,GeneID,Ranking
15852,ENSG00000253333.1,2
1791,ENSG00000200575.1,3
3223,ENSG00000163221.8,4


### Top 50

In [10]:
Top50.to_csv ('HYS1vs2_Boruta_Top50_Female.csv', index = False)
selected_rf_features.to_csv ('HYS1vs2_Boruta_All_Female.csv', index = False)

In [11]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top50_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3716\1748721246.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000236111.5,ENSG00000200575.1,ENSG00000163220.10,ENSG00000163221.8,ENSG00000202400.1,ENSG00000172046.18,ENSG00000004838.13,ENSG00000127418.14,ENSG00000253333.1,ENSG00000197043.13,...,ENSG00000260160.1,ENSG00000250507.1,ENSG00000276768.1,ENSG00000266104.1,ENSG00000099985.3,ENSG00000205853.10,ENSG00000100379.17,ENSG00000278948.1,ENSG00000231728.3,HYS
0,6,16,56086,1128,5,1866,52,153,4,7472,...,65,6,10,18,102,25,147,87,60,1
1,3,5,17851,449,0,1609,52,185,11,3471,...,35,13,4,17,109,24,100,95,26,2


### Top 100

In [12]:
Top100 = selected_rf_features.iloc[:100, :]
Top100.to_csv ('HYS1vs2_Boruta_Top100_Female.csv', index = False)
Top100.head(2)

,GeneID,Ranking
15852,ENSG00000253333.1,2
1791,ENSG00000200575.1,3


In [13]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top100_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3716\2034953418.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000157870.15,ENSG00000171824.13,ENSG00000236111.5,ENSG00000200575.1,ENSG00000270339.3,ENSG00000232671.5,ENSG00000163220.10,ENSG00000163221.8,ENSG00000233791.5,ENSG00000263528.7,...,ENSG00000124107.5,ENSG00000266104.1,ENSG00000099985.3,ENSG00000199248.1,ENSG00000205853.10,ENSG00000100379.17,ENSG00000278948.1,ENSG00000100359.20,ENSG00000231728.3,HYS
0,156,1120,6,16,86,14,56086,1128,87,1464,...,97,18,102,5,25,147,87,71,60,1
1,139,633,3,5,21,12,17851,449,28,992,...,62,17,109,5,24,100,95,68,26,2


### Top 250

In [14]:
Top250 = selected_rf_features.iloc[:250, :]
Top250.to_csv ('HYS1vs2_Boruta_Top250_Female.csv', index = False)
Top250.head(2)

,GeneID,Ranking
15852,ENSG00000253333.1,2
1791,ENSG00000200575.1,3


In [15]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top250_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3716\2988535486.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000186827.10,ENSG00000157870.15,ENSG00000215788.10,ENSG00000171824.13,ENSG00000255811.1,ENSG00000281207.1,ENSG00000178028.13,ENSG00000236111.5,ENSG00000200575.1,ENSG00000234184.5,...,ENSG00000102100.15,ENSG00000184205.14,ENSG00000276575.1,ENSG00000235262.1,ENSG00000231154.1,ENSG00000231728.3,ENSG00000129680.15,ENSG00000225840.2,ENSG00000228253.1,HYS
0,176,156,1362,1120,24,142,858,6,16,18,...,190,461,25,44,56,60,410,0,25,1
1,105,139,985,633,4,98,588,3,5,9,...,111,300,8,35,27,26,110,0,17,2


### Top 500

In [16]:
Top500 = selected_rf_features.iloc[:500, :]
Top500.to_csv ('HYS1vs2_Boruta_Top500_Female.csv', index = False)
Top500.head(2)

,GeneID,Ranking
15852,ENSG00000253333.1,2
1791,ENSG00000200575.1,3


In [17]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top500_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3716\2825327841.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000186827.10,ENSG00000107404.19,ENSG00000189409.13,ENSG00000067606.16,ENSG00000225931.3,ENSG00000157870.15,ENSG00000215788.10,ENSG00000228150.1,ENSG00000171824.13,ENSG00000116661.10,...,ENSG00000188917.14,ENSG00000231154.1,ENSG00000231728.3,ENSG00000233382.6,ENSG00000129680.15,ENSG00000089820.15,ENSG00000225008.1,ENSG00000225840.2,ENSG00000228253.1,HYS
0,176,541,18,508,47,156,1362,11,1120,70,...,1048,56,60,277,410,2805,16,0,25,1
1,105,335,20,374,72,139,985,7,633,48,...,825,27,26,143,110,2916,4,0,17,2
